# Thorium Calibration

We simulate a thorium mantle attached next to the tank. We calculate the expected spectrum for the energy deposition as well as the rate of incidence.

In [53]:
#Include modules
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from accelerate import parallel, concat
from tqdm.notebook import tqdm

%matplotlib widget

In [2]:
# Read the files

# Parameters
run                     = 'run3'
header                  = ['EventID','TrackID','Particle','EnergyDeposited','XPosition','YPosition','ZPosition','LocalTime','Volume']

# Find the files
foldername  = f'./data/{run}/output'
filenames   = os.listdir(foldername)

print(f'Found {len(filenames)} data files in {foldername}')

def loadFile(filename):
    return pd.read_csv(os.path.join(foldername,filename),skiprows=14,names=header)

loadFiles = parallel(loadFile)

if __name__ == '__main__':
    files = loadFiles(filenames)

Found 1024 data files in ./data/run3/output
100%|██████████| 1024/1024 [00:01<00:00, 1003.78it/s]


In [48]:
# Function to extract the energy deposited per track
def getEnergy(data):
    return data.groupby(['EventID','TrackID','Particle'])['EnergyDeposited'].sum().reset_index()

getEnergies = concat(parallel(getEnergy))

# Get the deposited energies
energy = getEnergies(files)

# Split into the energy per particle
particleEnergies = {particle:energy.loc[energy.Particle.str.contains(particle)] for particle in energy.Particle.unique()}



100%|██████████| 1024/1024 [00:01<00:00, 662.33it/s]


In [58]:
# Plot a histogram
fig = plt.figure(figsize=(10,5))
ax  = fig.add_subplot(111)
stackedEnergies = np.array([particleEnergies[particle].EnergyDeposited.to_list() for particle in particle_energies])
n,bins,_ = ax.hist(stackedEnergies, bins=700, stacked=True, density=True);

ax.set_xlim(bins[1],2)
ax.set_ylim(0,n[1])
ax.set_xlabel('Energy Deposited on TES Substrate during Thorium Runs [MeV]')
ax.set_ylabel('Density')
# ax.set_yscale('log')

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
NBINS       = 700
LIMS        = (0,2)
bins        = np.linspace(*LIMS,NBINS)
stackedHist = [np.histogram(particleEnergies[particle].EnergyDeposited,bins=) for particle in particle_energies]